In [5]:
import pandas as pd
import pickle
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


In [6]:
# Load features from a pickle file
feature_dict_path = "/home/suraj/Repositories/FM-extractors-radiomics/evaluation/features/rider_features.pkl" # Path to the pickle file containing features
with open(feature_dict_path, 'rb') as file: # Open the file in read binary mode
    data = pickle.load(file) # Load the data from the pickle file

In [13]:
similarity_scores = []

for model, df in data.items():
    data_list = []
    for idx, item in enumerate(df["train"]):
        row = item["row"]
        if item["feature"].shape[0] != 1:
            item["feature"] = np.mean(item["feature"], axis=0, keepdims=1)

        for i in range(item["feature"].shape[1]):
            row[f"feature_{i}"] = item["feature"][0, i]
        data_list.append(row)

    df = pd.DataFrame(data_list)
    df["Category"] = df["PatientID"].apply(lambda x: x.split("-")[-1])
    df["PatientID"] = df["PatientID"].apply(lambda x: x.split("-")[0])

    # Group by PatientID
    grouped = df.groupby("PatientID")

    row = {
        "Model": model
    }
    # List to store results
    similarity_results = []

    # Iterate over each group
    for patient_id, group in grouped:
        # Separate test and retest
        test_features = group[group["Category"] == "test"].filter(like="feature").values
        retest_features = group[group["Category"] == "retest"].filter(like="feature").values

        # Calculate cosine similarity if both test and retest are available
        if test_features.size > 0 and retest_features.size > 0:
            similarity = cosine_similarity(test_features, retest_features)
            similarity_results.append({
                "PatientID": patient_id,
                "CosineSimilarity": similarity.mean() # Average similarity
            })

        row.update({patient_id: similarity.mean()})

    similarity_scores.append(row)

In [14]:
similarity_df = pd.DataFrame(similarity_scores)
similarity_df

,Model,1129164940,1225316081,1286684383,1332496276,1344011628,1346917136,1500037140,1532432635,1760553574,...,2619750334,2655999012,2669524182,2799584460,3023568408,3115188676,3152132495,3160137230,6324870494,9763310455
0,FMCIBExtractor,0.984755,0.991311,0.993948,0.988226,0.987306,0.992159,0.994090,0.990584,0.986328,...,0.992554,0.994005,0.992305,0.996201,0.986543,0.981461,0.989918,0.992072,0.987163,0.988515
1,CTFMExtractor,0.980054,0.972268,0.986338,0.976405,0.973627,0.984043,0.980602,0.989095,0.986237,...,0.972104,0.990409,0.984938,0.994559,0.990190,0.985550,0.965330,0.985030,0.970803,0.990520
2,CTClipVitExtractor,0.690198,0.966613,0.967126,0.979720,0.969774,0.985830,0.976854,0.987476,0.989507,...,0.990872,0.479299,0.904442,0.978727,0.879736,0.969456,0.935896,0.983428,0.924488,0.987791
3,PASTAExtractor,0.988589,0.984902,0.992129,0.994269,0.980483,0.988990,0.993281,0.986486,0.994789,...,0.982148,0.991020,0.994971,0.994306,0.991442,0.991297,0.990841,0.987193,0.990207,0.992046
4,VISTA3DExtractor,0.952474,0.942863,0.981147,0.970828,0.957722,0.959806,0.972630,0.962932,0.979448,...,0.972161,0.974797,0.973922,0.989251,0.972537,0.969385,0.960788,0.979471,0.960681,0.973082
5,VocoExtractor,0.999998,0.999992,0.999999,0.999993,0.999999,1.000000,1.000000,0.999995,1.000000,...,1.000000,0.999998,1.000000,0.999998,0.999998,0.999999,0.999997,1.000000,1.000000,0.999992
6,SUPREMExtractor,0.999454,0.998917,0.999582,0.999613,0.999423,0.999705,0.999516,0.999499,0.999667,...,0.999697,0.999494,0.999625,0.999662,0.999415,0.999435,0.999409,0.999566,0.999421,0.999711
7,MerlinExtractor,0.797546,0.756355,0.739302,0.831392,0.924906,0.854537,0.878959,0.796966,0.417270,...,0.885625,0.879914,0.825201,0.842104,0.908874,0.782401,0.893829,0.929624,0.833247,0.842944
8,MedImageInsightExtractor,0.997524,0.998003,0.998669,0.998706,0.998494,0.998673,0.998069,0.998142,0.998220,...,0.996315,0.997779,0.997982,0.998881,0.997696,0.996069,0.998863,0.998862,0.997702,0.996759
9,ModelsGenExtractor,0.961621,0.964430,0.983384,0.983220,0.972619,0.984502,0.983838,0.984862,0.987475,...,0.985298,0.991589,0.985709,0.992849,0.979574,0.968933,0.978932,0.982646,0.972922,0.989212


In [50]:
import plotly.express as px

# Calculate the average similarity for each model
similarity_df['AverageSimilarity'] = similarity_df.iloc[:, 1:].mean(axis=1)

# Create a bar plot using the 'simple_white' template with specific dimensions
fig = px.bar(similarity_df, x='Model', y='AverageSimilarity', 
             title='', template='simple_white', width=800, height=1200, range_y=(0.6,1))

# Update layout for a minimalist appearance similar to file_context_0
fig.update_layout(
    margin=dict(l=20, r=20, t=40, b=20),
    xaxis=dict(showline=False, showgrid=False, tickfont=dict(size=30), title='Model'),
    yaxis=dict(showgrid=False, tickfont=dict(size=30), title='Average Similarity'),
    title=dict(x=0.5, xanchor='center'),
    font=dict(size=30, family="Arial"),
    width=800,
    height=1200,
    showlegend=False,
)

# Update traces with a different color
fig.update_traces(
    marker_color="purple",  # Using purple for marker color
    marker_opacity=0.6,
)

# Add text annotations for each bar to display its average similarity value
for idx, row in similarity_df.iterrows():
    fig.add_annotation(
        x=row['Model'],
        y=row['AverageSimilarity'],
        text=f"{row['AverageSimilarity']:.2f}",
        showarrow=False,
        yshift=15,
        font=dict(size=25, color="black")
    )

# Show the plot
fig.show()
